# Run Models Notebook

This notebook demonstrates executing code and uploading results to GitHub from Google Cloud Run.

In [ ]:
import os
import requests
import base64
import json
from datetime import datetime
from google.cloud import secretmanager

In [ ]:
# Get the GitHub token from Secret Manager
def get_github_token():
    try:
        client = secretmanager.SecretManagerServiceClient()
        name = f"projects/{os.environ.get('GOOGLE_CLOUD_PROJECT')}/secrets/github-token/versions/latest"
        response = client.access_secret_version(request={"name": name})
        return response.payload.data.decode("UTF-8")
    except Exception as e:
        print(f"Error getting GitHub token: {e}")
        return None

GITHUB_TOKEN = get_github_token()
print(f"GitHub token available: {GITHUB_TOKEN is not None}")

In [ ]:
def upload_reports_to_github(repo, token, branch='main', commit_message='Report from notebook execution'):
    if not token:
        print("No GitHub token available")
        return
    
    # Create a simple report file
    report_content = f"""
# Execution Report

Generated: {datetime.now().isoformat()}
Environment: Google Cloud Run
Project: {os.environ.get('GOOGLE_CLOUD_PROJECT', 'unknown')}
Status: Success
"""
    
    headers = {
        'Authorization': f'token {token}',
        'Accept': 'application/vnd.github.v3+json'
    }
    
    file_path = f"reports/execution-{datetime.now().strftime('%Y%m%d-%H%M%S')}.md"
    content_encoded = base64.b64encode(report_content.encode('utf-8')).decode('utf-8')
    
    data = {
        'message': commit_message,
        'content': content_encoded,
        'branch': branch
    }
    
    url = f'https://api.github.com/repos/{repo}/contents/{file_path}'
    
    try:
        response = requests.put(url, headers=headers, json=data)
        
        if response.status_code == 201:
            result = response.json()
            print(f"Successfully uploaded report to GitHub!")
            print(f"File: {file_path}")
            return result
        else:
            print(f"Failed to upload. Status: {response.status_code}")
            return None
            
    except Exception as e:
        print(f"Error uploading to GitHub: {e}")
        return None

In [ ]:
print("Starting notebook execution...")
print("Notebook executed successfully!")

In [ ]:
# Upload results to GitHub
TARGET_REPO = "modelearth/reports"

if GITHUB_TOKEN:
    print("Uploading results to GitHub...")
    result = upload_reports_to_github(
        repo=TARGET_REPO,
        token=GITHUB_TOKEN,
        branch='main',
        commit_message='Automated report from Cloud Run'
    )
    
    if result:
        print("Execution completed successfully!")
    else:
        print("Notebook executed but upload failed")
else:
    print("No GitHub token available - skipping upload")